In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyer_loader import JupyerLoader

loader = JupyerLoader()
loader.load_all()

# Prototype 1 

In the last chapters many ideas have been introduced and the engine was successively improved. One concesequence though is that the necessary code is split across multiple files. The goal of this chapter is to show one complete implementation of the current engine. Additionally some last improvement is done to the `play` function. Up to now it was just using `analyse` internally and would calculate an exact score for every possible move. But as we are only interested in the best move, there is some room for improvement here.

We define a new engine `PrototypeV1Engine` that inherits the base interface `Engine`. It will contain the complete implementation of the algorithms introduced in the previous chapters, which will not be explained again here.

In [ ]:
from converted_notebooks.s04_engine_interface import Engine, ScoredMove
from converted_notebooks.s12_simplified_evaluation import DetailedMove, IncrementalEvaluator
from converted_notebooks.s11_iterative_deepening import AlphaBetaCache, NodeType, cache_alpha_beta

import chess
import random


class PrototypeV1Engine(Engine):
    PLAYER_MULTIPLIER = {
        chess.WHITE: 1, chess.BLACK: -1
    }

    def __init__(self, evaluator: IncrementalEvaluator, max_look_ahead_depth):
        self.evaluator = evaluator
        self.max_look_ahead_depth = max_look_ahead_depth
        self.cache = AlphaBetaCache()

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> int:
        if (score := self.evaluator.evaluate(board)) is not None:
            return score
        if depth == 0:
            return self._quiescence(board, alpha, beta)

        for move in board.legal_moves:
            detailedMove = DetailedMove(board, move)

            self.evaluator.push(detailedMove)
            board.push(move)
            value = -1 * self._value(board, depth - 1, -beta, -alpha)
            board.pop()
            self.evaluator.pop()

            if value >= beta:
                return value
            alpha = max(alpha, value)

        return alpha

    def _quiescence(self, board: chess.Board, alpha: int, beta: int) -> int:
        stand_pat = self.evaluator.get_score()

        if stand_pat >= beta:
            return beta
        alpha = max(alpha, stand_pat)

        for move in board.generate_legal_captures():
            detailedMove = DetailedMove(board, move)

            if move.promotion is None:
                if self._canDeltaPrune(stand_pat, alpha, detailedMove):
                    continue
                if self._seeCapture(board, detailedMove) < 0:
                    continue

            self.evaluator.push(detailedMove)
            board.push(move)
            value = -1 * self._quiescence(board, -beta, -alpha)
            board.pop()
            self.evaluator.pop()

            if value >= beta:
                return value
            alpha = max(alpha, value)
        return alpha

    def _canDeltaPrune(
        self, stand_pat: int, alpha: int, detailedMove: DetailedMove
    ):
        pieceValue = self.evaluator.piece_values[
            detailedMove.capturedPiece.piece.piece_type]
        bestAlpha = stand_pat + pieceValue + 200
        return bestAlpha < alpha

    def _seeCapture(self, board: chess.Board, detailedMove: DetailedMove):
        board.push(detailedMove.move)
        capturedPiece = detailedMove.capturedPiece
        value = self.evaluator.piece_values[
            capturedPiece.piece.piece_type
        ] - self._see(board, detailedMove.placedPiece.square)
        board.pop()
        return value

    def _see(self, board: chess.Board, square: chess.Square):
        attackers_squares = [
            square for square in board.attackers(board.turn, square)
            if not board.is_pinned(board.turn, square)
        ]
        if not attackers_squares:
            return 0

        attacker_square = min(
            attackers_squares,
            key=lambda attackers_square: board.piece_type_at(attackers_square)
        )
        capturedPiece = board.piece_type_at(square)

        detailedMove = DetailedMove(
            board, chess.Move(from_square=attacker_square, to_square=square)
        )
        board.push(detailedMove.move)
        value = max(
            0,
            self.evaluator.piece_values[capturedPiece] -
            self._see(board, detailedMove.placedPiece.square)
        )

        board.pop()
        return value

Next, the `analyse` methhod is implemented. The goal of this method is to evaluate all next moves and return them as a sorted list. To implement this two function `_evaluate_nove` and `_evaluate_moves` are needed. All of these have been shown already in previous chapters, so there will be just short descriptions for a reminder.

The `_evaluate_move` function will score the given move by using the `_value` function.

In [ ]:
def _evaluate_move(
    self, board: chess.Board, move: chess.Move, depth: int
) -> ScoredMove:
    self.evaluator.push(DetailedMove(board, move))
    board.push(move)

    score = self._value(
        board,
        depth,
        -self.evaluator.value_checkmate - 1,
        self.evaluator.value_checkmate + 1
    )
    score *= self.PLAYER_MULTIPLIER[board.turn]

    board.pop()
    self.evaluator.pop()

    return ScoredMove(score=score, move=move)


PrototypeV1Engine._evaluate_move = _evaluate_move

The `_evaluate_moves` implement the iterative deepening algorithm and thus will score all moves with the `_evaluate_move` method starting at a depth of 0. It will then successively increase the depth until the max look ahead depth is reached. 

In [ ]:
def _evaluate_moves(self, board: chess.Board) -> list[ScoredMove]:
    print(f"Max depth: {self.max_look_ahead_depth}")
    self.cache.clear()
    self.evaluator.init(board)

    for depth in range(self.max_look_ahead_depth + 1):
        scoredMoves = [
            self._evaluate_move(board, move, depth)
            for move in board.legal_moves
        ]

        print(f"Depth {depth}")

    return scoredMoves


PrototypeV1Engine._evaluate_moves = _evaluate_moves

Then `analyse` uses this method to get a list of scored moves and will return this sorted to the caller.

In [ ]:
def analyse(self, board: chess.Board) -> list[ScoredMove]:
    nextMoves = self._evaluate_moves(board.copy(stack=False))

    whitesTurn = board.turn is chess.WHITE
    nextMoves.sort(reverse=whitesTurn)

    return nextMoves


PrototypeV1Engine.analyse = analyse

Besides `analyse` the interface also requires us to implement the `play` function. In contrast to `analyse`, `play` should only return the best move. Therefore the iterative deepening algorithm could call directly the `_value` function. The problem though is that `_value` gives us only the best score, but not the corresponding move. There are many possible solutions to this problem. One of them is to rewrite `_value` so it actually returns the move. This implemention instead makes use of the cache to find the best move afterwards. 

The `_find_move` method takes a chess board `board` and the score `score` of the best move as parameters. Its goal is to find a corresponding move by querying the cache and return it. It will therefore iterate over all moves and get the cache entries for these if existent. To be the best move the cache entry type needs to be exact and the stored value must be the same as score negated. The negation is needed because the cache entry is for a position one move further than the passed board and therefore the perspective changed. There can be multiple best moves, therefore they are saved in a list and a random element is returned from it in the end.

In [ ]:
def _find_move(self, board: chess.Board, score: int) -> chess.Move:
    best_moves = []
    for move in board.legal_moves:
        board.push(move)
        cacheKey = self.cache.get_key(board)
        board.pop()

        try:
            type, value = self.cache.load_cache(cacheKey, self.max_look_ahead_depth - 1)

            if type == NodeType.EXACT and value == -score:
                best_moves.append(move)
        except KeyError:
            pass

    assert best_moves, "No best move found with the given score"
    return random.choice(best_moves)


PrototypeV1Engine._find_move = _find_move

No we can implement the iterative deepening algorithm for `play`. The method `_find_best_move` works similary to `_evaluate_moves`. It will also take the chess board `board` as a parameter. But instead of returning a list of all scored moves, it will just return the best move. To do this it calls directly `_value` in its loop. At the end it uses the previously defined `_find_move` method to get the best move from the cache.

In [ ]:
def _find_best_move(self, board: chess.Board) -> chess.Move:
    print(f"Max depth: {self.max_look_ahead_depth}")
    self.cache.clear()
    self.evaluator.init(board)

    for depth in range(self.max_look_ahead_depth + 1):
        score = self._value(
            board,
            depth,
            -self.evaluator.value_checkmate - 1,
            self.evaluator.value_checkmate + 1
        )
        print(f"Depth {depth}")

    return self._find_move(board, score)


PrototypeV1Engine._find_best_move = _find_best_move

Lastly, the `play` method needs to be implemented. It will simply return a `chess.engine.PlayResult` with the best move found by the `_find_best_move` method.

In [ ]:
def play(self, board: chess.Board) -> chess.engine.PlayResult:
    return chess.engine.PlayResult(
        move=self._find_best_move(board), ponder=None
    )


PrototypeV1Engine.play = play

It is then possible to play against the prototype engine with the human engine.

In [ ]:
# import random
# from converted_notebooks.s06_play import play_game
# from converted_notebooks.s07_human_engine import HumanEngine
# from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

# random.seed(42)

# board = chess.Board()
# play_game(
#     board,
#     HumanEngine(),
#     PrototypeV1Engine(evaluator=incremental_simplified_evaluator, max_look_ahead_depth=3),
#     display_board=True,
#     log_moves=True
# )
# print(board.outcome())